In [ ]:
class MoodBasedRecommender:
    def __init__(self, book_profiler):
        self.book_profiler = book_profiler

    def calculate_similarity(self, mood_vector, book_vector):
        """
        Calculate cosine similarity between mood and book emotion vectors
        """
        # Convert to numpy arrays
        mood_array = np.array(list(mood_vector.values()))
        book_array = np.array(list(book_vector.values()))

        # Calculate cosine similarity
        dot_product = np.dot(mood_array, book_array)
        mood_norm = np.linalg.norm(mood_array)
        book_norm = np.linalg.norm(book_array)

        if mood_norm == 0 or book_norm == 0:
            return 0

        similarity = dot_product / (mood_norm * book_norm)
        return similarity

    def recommend_by_mood(self, user_mood, top_n=5, mode='match'):
        """
        Recommend books based on user's current mood

        Parameters:
        - user_mood: dict with emotion names as keys and weights as values
        - top_n: number of recommendations to return
        - mode: 'match' (similar emotions) or 'contrast' (opposite emotions)
        """
        recommendations = []

        # Normalize user mood vector
        all_emotions = list(emotion_lexicon.keys())
        mood_vector = {emotion: user_mood.get(emotion, 0) for emotion in all_emotions}

        # Calculate similarity for each book
        for book_id, profile in self.book_profiler.book_profiles.items():
            book_vector = profile['emotion_scores']

            similarity = self.calculate_similarity(mood_vector, book_vector)

            # For contrast mode, invert the similarity
            if mode == 'contrast':
                similarity = 1 - similarity

            recommendations.append({
                'book_id': book_id,
                'title': profile['title'],
                'similarity': similarity,
                'top_emotions': sorted(book_vector.items(), key=lambda x: x[1], reverse=True)[:3]
            })

        # Sort by similarity and return top N
        recommendations.sort(key=lambda x: x['similarity'], reverse=True)
        return recommendations[:top_n]

    def recommend_by_text(self, mood_text, top_n=5, mode='match'):
        """
        Recommend books based on user's mood described in text
        """
        # Extract emotions from mood text
        mood_emotions = emotion_extractor.extract_emotions(mood_text)
        user_mood = mood_emotions['scores']

        return self.recommend_by_mood(user_mood, top_n, mode)

    def display_recommendations(self, recommendations, mode='match'):
        """
        Display recommendations in a nice format
        """
        mode_text = "matching your mood" if mode == 'match' else "to contrast your mood"
        print(f"\n{'='*70}")
        print(f"📚 TOP BOOK RECOMMENDATIONS {mode_text.upper()}")
        print(f"{'='*70}\n")

        for i, rec in enumerate(recommendations, 1):
            print(f"{i}. {rec['title']}")
            print(f"   Match Score: {rec['similarity']:.2%}")
            print(f"   Primary Emotions: {', '.join([f'{e[0]}' for e in rec['top_emotions']])}")
            print()

# Initialize recommender
recommender = MoodBasedRecommender(book_profiler)
print("✓ Mood-based recommender initialized")

In [ ]:
# Test 1: User feeling happy and excited
print("\n🎭 SCENARIO 1: User is feeling happy and excited")
mood_text = "I'm feeling so happy and excited today! I want something joyful and uplifting."
recommendations = recommender.recommend_by_text(mood_text, top_n=3, mode='match')
recommender.display_recommendations(recommendations, mode='match')

In [ ]:
# Test 2: User feeling sad and wants contrast
print("\n🎭 SCENARIO 2: User is feeling sad and wants something different")
mood_text = "I'm feeling really sad and down today. I want something to cheer me up."
recommendations = recommender.recommend_by_text(mood_text, top_n=3, mode='contrast')
recommender.display_recommendations(recommendations, mode='contrast')

In [ ]:
# Test 3: User wants adventure
print("\n🎭 SCENARIO 3: User wants adventure and excitement")
mood_text = "I'm in the mood for an exciting adventure! Something thrilling and action-packed."
recommendations = recommender.recommend_by_text(mood_text, top_n=3, mode='match')
recommender.display_recommendations(recommendations, mode='match')

In [ ]:
# Test 4: User wants romance
print("\n🎭 SCENARIO 4: User wants romance")
user_mood = {
    'romantic': 80,
    'happy': 60,
    'peaceful': 40
}
recommendations = recommender.recommend_by_mood(user_mood, top_n=3, mode='match')
recommender.display_recommendations(recommendations, mode='match')